# TFIDF Implementation on a corpus 

Here we have implemented TFIDF vectorization on a test corpus. The first task is implementation of TFIDF and the second task considers TFIDF implementation to get top features only.

### 1. Use all features

### Corpus

In [2]:

corpus = [
     'this is the first document',
     'this document is the second document',
     'and this is the third one',
     'is this the first document',
]

In [3]:
# Import libraries 
from collections import Counter
from tqdm import tqdm
from scipy.sparse import csr_matrix
import math
import operator
from sklearn.preprocessing import normalize
import numpy

In [14]:
# Reference for fit and transform methods: Assignment_3_Reference.ipynb from https://www.appliedaicourse.com
class tfidf_implement():
  ''' Implementation of tfidf '''
  def __init__(self):
    vocab = []
    idf_values = []

  def fit(self, dataset):
     ''' Fit dataset - takes dataset as input, returns dictionary 
     of unique words and respective indices from whole dataset in alphabetical order '''
     if isinstance(dataset, list):
      unique_words = set()
      for row in dataset:
        word_list = row.split(" ")
        for word in word_list:
          if len(word)<2:
            continue
          unique_words.add(word)
      sort_words = sorted(list(unique_words))
      # Stored in __init__ so it can be used by other methods and for ease of getting vocab when user wants them
      self.vocab = {j:i for i,j in enumerate(sort_words)}
      return
     else:
      print("Please give list of strings")

  def idf(self, dataset):
    ''' Compute idf values of all words returned by fit. Returns idf values and 
    order of idf values is maintained as per dictionary of fit\n 
    Note: idf uses dictionary of fit, so should run fit first'''
    # Split sentences, list of words are taken
    words = [[word for word in row.split(" ")] for row in dataset]
    idf_values = []
    # for every word in vocab
    for t in self.vocab:
      count = 0
      # for every row of split dataset, words
      for doc in words:
        # if vocab word is present in that row of dataset
        if t in doc:
          count += 1
      # idf is calculated
      idf = 1 + math.log((1+len(dataset))/(1+count))
      # idf values are appended 
      idf_values.append(idf)
    # Stored to be accessed easily by other methods in class and for ease of getting idf_values when user wants them
    self.idf_values = idf_values
    return
      
  def transform(self, dataset):
    ''' Input dataset, returns sparse matrix of tfidf values\n
    Note: Should run fit and idf functions before running transform '''
    rows = []
    columns = []
    values = []
    if isinstance(dataset, list):
      for idx, row in enumerate(tqdm(dataset)):
        row = row.split(" ")
        word_list = dict(Counter(row))
        for word, freq in word_list.items():
          if len(word)<2:
            continue
          col_idx = self.vocab.get(word, -1)
          if col_idx !=-1:
            rows.append(idx)
            columns.append(col_idx)
            # tfidf values are computed
            # idf_values has same index as taken from vocab's values, since its values are appended wrt vocab words
            val = (freq/len(row)) * self.idf_values[col_idx]
            values.append(val)
      return normalize(csr_matrix((values,(rows,columns)), shape=(len(dataset), len(self.vocab))))


In [15]:
c1 = tfidf_implement()
c1.fit(corpus)
c1.idf(corpus)

print(c1.transform(corpus).toarray()[0])

100%|██████████| 4/4 [00:00<00:00, 5704.60it/s]

[0.         0.46979139 0.58028582 0.38408524 0.         0.
 0.38408524 0.         0.38408524]


In [16]:
c1.vocab

{'and': 0,
 'document': 1,
 'first': 2,
 'is': 3,
 'one': 4,
 'second': 5,
 'the': 6,
 'third': 7,
 'this': 8}

In [17]:
# idf values 
c1.idf_values

[1.916290731874155,
 1.2231435513142097,
 1.5108256237659907,
 1.0,
 1.916290731874155,
 1.916290731874155,
 1.0,
 1.916290731874155,
 1.0]

### 2. Select only top features 

In [20]:
# Below is the code to load the cleaned_strings pickle file provided
# Here corpus is of list type

import pickle
with open('test_corpus.txt', 'rb') as f:
    corpus = pickle.load(f)
    
# printing the length of the corpus loaded
print("Number of documents in corpus = ",len(corpus))

Number of documents in corpus =  746


In [21]:
# Ref_1 : https://stackoverflow.com/questions/29216889/slicing-a-dictionary
# Ref_2 : https://stackoverflow.com/questions/9919604/efficiently-calculate-word-frequency-in-a-string

# Same as above class, only difference here is a new method pickTop(n) which gives top n words based on largest idf values

class tfidf_implement2():
  ''' Implementation 2 of tfidf '''

  def __init__(self):
    vocab = []
    idf_values = []

  def fit(self, dataset):
    ''' Fit dataset - takes dataset as input, returns dictionary 
    of unique words and respective indices from whole dataset in alphabetical order '''
    if isinstance(dataset, list):
      unique_words = set()
      for row in dataset:
        word_list = row.split(" ")
        for word in word_list:
          if len(word)<2:
            continue
          unique_words.add(word)
      sort_words = sorted(list(unique_words))
      self.vocab = {j:i for i,j in enumerate(sort_words)}
      return self.vocab
    else:
      print("Please give list of strings")

  def idf(self, dataset):
    ''' Compute idf values of all words returned by fit. Returns idf values and 
    order of idf values is maintained as per dictionary of fit\n 
    Note: idf uses dictionary of fit, so should run fit first'''
    words = [[word for word in row.split(" ")] for row in dataset]
    idf_values = []
    for t in self.vocab:
      count = 0
      for doc in words:
        if t in doc:
          count += 1
      idf = 1 + math.log((1+len(dataset))/(1+count))
      idf_values.append(idf)
    self.idf_values = idf_values
    return
  
  def pickTop(self,n):
    ''' Returs none, Picks top 'n' features based on idf values\n 
    Note: fit and idf should be ran first'''
    # Take numpy array of idf_values and call argsort to get indices of n largest idf values
    idf = numpy.array(self.idf_values)
    sort_idx = list(numpy.argsort(idf))[-n:]
    # initialize new empty vocab and idf_values
    new_vocab = []
    new_idf = []
    i = 0
    # for every (key,value) pair in vocab
    for item in self.vocab.items():
      # if value of item, i.e index of that vocab is in list of sorted indices 
      if item[1] in sort_idx:
        # append idf_value corresponding to the index in vocab
        new_idf.append(idf[item[1]])
        # Append tupple with same key and value is the new index 
        new_vocab.append((item[0],i))
        i = i+1
    # Overwrite vocab and idf_values in __init__ to be used for further methods and ease of getting values if user wants
    self.vocab = dict(new_vocab)
    self.idf_values = new_idf

  def transform(self, dataset):
    ''' Input dataset, returns sparse matrix of tfidf values\n
    Note: Should run fit and idf functions before running transform '''
    rows = []
    columns = []
    values = []
    if isinstance(dataset, list):
      for idx, row in enumerate(tqdm(dataset)):
        row = row.split(" ")
        word_list = dict(Counter(row))
        for word, freq in word_list.items():
          if len(word)<2:
            continue
          col_idx = self.vocab.get(word, -1)
          if col_idx !=-1:
            rows.append(idx)
            columns.append(col_idx)
            val = (freq/len(row)) * self.idf_values[col_idx]
            values.append(val)
      return normalize(csr_matrix((values,(rows,columns)), shape=(len(dataset), len(self.vocab))))


In [22]:
c2 = tfidf_implement2()
c2.fit(corpus)
c2.idf(corpus)
c2.pickTop(50)
output = c2.transform(corpus)

100%|██████████| 746/746 [00:00<00:00, 102915.86it/s]


In [23]:
c2.vocab

{'gone': 0,
 'gosh': 1,
 'goth': 2,
 'gotta': 3,
 'gotten': 4,
 'government': 5,
 'grade': 6,
 'gradually': 7,
 'grainy': 8,
 'granted': 9,
 'graphics': 10,
 'grasp': 11,
 'grates': 12,
 'halfway': 13,
 'ham': 14,
 'handle': 15,
 'handles': 16,
 'hang': 17,
 'hankies': 18,
 'hanks': 19,
 'happiness': 20,
 'happy': 21,
 'harris': 22,
 'hatred': 23,
 'havilland': 24,
 'hay': 25,
 'hayao': 26,
 'hayworth': 27,
 'hbo': 28,
 'heads': 29,
 'hearts': 30,
 'heartwarming': 31,
 'heche': 32,
 'heels': 33,
 'heist': 34,
 'helen': 35,
 'hellish': 36,
 'helms': 37,
 'help': 38,
 'helping': 39,
 'hendrikson': 40,
 'hernandez': 41,
 'hero': 42,
 'heroes': 43,
 'heroine': 44,
 'heroism': 45,
 'hes': 46,
 'hide': 47,
 'higher': 48,
 'zombiez': 49}

In [24]:
print(output.shape)

(746, 50)


In [27]:
c2.idf_values

[6.922918004572872,
 6.922918004572872,
 6.922918004572872,
 6.922918004572872,
 6.922918004572872,
 6.922918004572872,
 6.922918004572872,
 6.922918004572872,
 6.922918004572872,
 6.922918004572872,
 6.922918004572872,
 6.922918004572872,
 6.922918004572872,
 6.922918004572872,
 6.922918004572872,
 6.922918004572872,
 6.922918004572872,
 6.922918004572872,
 6.922918004572872,
 6.922918004572872,
 6.922918004572872,
 6.922918004572872,
 6.922918004572872,
 6.922918004572872,
 6.922918004572872,
 6.922918004572872,
 6.922918004572872,
 6.922918004572872,
 6.922918004572872,
 6.922918004572872,
 6.922918004572872,
 6.922918004572872,
 6.922918004572872,
 6.922918004572872,
 6.922918004572872,
 6.922918004572872,
 6.922918004572872,
 6.922918004572872,
 6.922918004572872,
 6.922918004572872,
 6.922918004572872,
 6.922918004572872,
 6.922918004572872,
 6.922918004572872,
 6.922918004572872,
 6.922918004572872,
 6.922918004572872,
 6.922918004572872,
 6.922918004572872,
 6.922918004572872]